In [63]:
import pandas as pd
import pm4py
import pm4py.read
import category_encoders as ce

In [93]:
ocel_in_file = "../../../data/CS/source/cs_january_ots[krs_krv_cv]_eas[ea2_ea4_resource_robot_resource_person_resource_multi].jsonocel"
objects_table_in_file = "../../../data/CS/source/objects.csv"

oft_out_file = "../../../data/CS/feature_encodings/baselines/OFT/objects.csv"

In [39]:
# load OCEL
ocel = pm4py.read.read_ocel(ocel_in_file)
objects_df = pd.read_csv(objects_table_in_file, sep=";")

In [ ]:
# select events that contain the target variable 'ea4'
events_to_take_target_from = (
    ocel.relations.sort_values("ocel:timestamp", ascending=False)
    .groupby("ocel:oid")
    .tail(1)[["ocel:eid", "ocel:oid"]]
)
# select the target variables for those events
events_with_target = ocel.events.loc[
    ocel.events["ocel:eid"].isin(events_to_take_target_from["ocel:eid"]),
    ["ocel:eid", "event_ea4"],
]
# select the objects relating to those events and their target variables
objects_with_target = events_to_take_target_from.join(
    events_with_target.set_index("ocel:eid"), on="ocel:eid"
).drop(columns=["ocel:eid"])
# join full objects table with these objects of which we now have target variable 'ea4'
objects_df_with_target = objects_with_target.join(
    objects_df.set_index("oid"), on="ocel:oid"
)

In [60]:
objects_df_with_target.head()

,ocel:oid,event_ea4,oa1,oa2,oa3,oa4,oa5,oa6,oa7,oa8,oa9,oa10,oa11,oa12,oa13,oa14,oa15,oa16
3002312,KRV-6108394,0.0,0.368881,Acceptatie,Vervolgonderzoek,NaN,-2.565395,Keten is niet langs Mid-office geweest,Keten is niet langs CLCS geweest,NaN,Dossier is afgerond,Particulieren,Person,Laag,Prospect,NaN,Laag,Afgehandeld
3002298,KRS-4344502,0.0,-1.129970,Acceptatie,KSS Acceptatie Dossier,NaN,-2.565395,Keten is niet langs Mid-office geweest,Keten is niet langs CLCS geweest,NaN,Dossier is afgerond,Grootzakelijk,Organisation,Laag,Prospect,NaN,NaN,Afgehandeld
3002282,KRV-1872351,3.0,0.368881,Event gedreven review,Vervolgonderzoek,NaN,-0.481001,Keten is niet langs Mid-office geweest,Keten is niet langs CLCS geweest,NaN,Dossier is afgerond,Midden- en kleinbedrijf 2,Organisation,Midden,Klant,NaN,Midden,Afgehandeld
3002280,KRS-1814915,0.0,0.368881,Event gedreven review,KSS BaU Signal,NaN,1.082294,Keten is niet langs Mid-office geweest,Keten is niet langs CLCS geweest,NaN,Dossier is afgerond,NaN,Person,Laag,Klant,NaN,NaN,Afgehandeld
3002279,KRV-9789608,0.0,0.368881,Event gedreven review,Vervolgonderzoek,NaN,1.082294,Keten is niet langs Mid-office geweest,Keten is niet langs CLCS geweest,NaN,Dossier is afgerond,NaN,Person,Laag,Klant,NaN,Laag,Afgehandeld


In [78]:
cat_cols = [k for (k,v) in objects_df_with_target.dtypes.to_dict().items() if v == object]
cat_cols.remove('ocel:oid')

In [89]:
X_raw, y = (
    objects_df_with_target.drop(columns=["event_ea4"]),
    objects_df_with_target.loc[:, "event_ea4"],
)
encoder = ce.CatBoostEncoder(cols=cat_cols)
X = encoder.fit_transform(X_raw, y)
X['ea4'] = y


In [94]:
X.to_csv(oft_out_file, index=False)